In [2]:
import numpy as np 
import pandas as pd 
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/fasttext-twitter-derived-embeddings/twitter_derived_embeddings
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv
/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/wikinews300d1mvec/wiki-news-300d-1M.vec


# Load Files

In [ ]:
#sub=pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
path = "/kaggle/input/nlp-getting-started/"
train = pd.read_csv(f'{path}train.csv')
test = pd.read_csv(f'{path}test.csv')

In [ ]:
test['text'].head(20)

In [ ]:
def remove_space(text):
    regex=re.compile(r'%\S?20')
    return regex.sub(r' ',text)

In [ ]:
def append_location(x):
    return ' '.join(train['location'].iloc[train.index[x]]+x)

In [ ]:
train['keyword'].fillna(value="None",inplace=True)
train['keyword']=train['keyword'].apply(lambda x :remove_space(x))
#train['text']=train['keyword']+'is location'+train['text']
train_id=train['id']
train.drop(columns=['id','keyword','location'],axis=1,inplace=True)
train['target']='__label__'+''+train['target'].astype(str)

test['keyword'].fillna(value="None",inplace=True)
test['keyword']=test['keyword'].apply(lambda x :remove_space(x))
#test['text']=test['keyword']+'is location '+test['text']
test_id=test['id']
test.drop(columns=['keyword','location'],axis=1,inplace=True)

In [ ]:
train['target']=train['target'].str.replace('1','disaster')
train['target']=train['target'].str.replace('0','notadisaster')


In [ ]:
#train['text']=train.text.str.encode('utf-8')
#train['target']=train.target.str.encode('utf-8')
train=train[['target','text']]

In [ ]:
train.shape,test.shape

In [ ]:
train.to_csv(r'train.txt', index=False, sep=' ',header=False)
test.to_csv(r'test.txt', index=False, sep=' ',header=False)

In [ ]:
n = (train.shape[0] * 7)/10
n = int(round(n))

# Split the file into 70% train and 30% test
fft_train = train[:n] 
fft_test = train[n:] 
print(fft_train.shape, fft_test.shape)
fft_train.to_csv(r'fft_train.txt', index=False, sep=' ', header=False)
fft_test.to_csv(r'fft_test.txt', index=False, sep=' ', header=False)

In [ ]:
np=fft_train['text'].values
length=[]
for i in np:
    length.append(len(i))
from statistics import mean
print('Mean no. of words',mean(length))

In [ ]:
import fasttext
import fasttext.util

In [ ]:
ftmodel = fasttext.train_supervised(input='/kaggle/working/train.txt',label_prefix="__label__",neg=5,epoch=10,dim=300,loss='hs',word_ngrams=2,ws=4,minn=2,maxn=6,pretrainedVectors='/kaggle/input/wikinews300d1mvec/wiki-news-300d-1M.vec')

In [ ]:
'''
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))
'''
ftmodel.test('/kaggle/working/train.txt')

In [ ]:

ftmodel.predict('Ohh!! #safetyfirst what an earthquake')[0][0]

In [ ]:
ftmodel.predict('today is a good day')[0][0]

In [ ]:
submit = test[['id','text']]
targets=[]

In [ ]:
submit['text']

In [ ]:
pred_label=ftmodel.predict(submit['text'][0])
pred_label=ftmodel.predict(submit['text'][345])
pred_label

In [ ]:
pred_label=ftmodel.predict(submit['text'][3125])
pred_label[0][0]

In [ ]:
submit.shape

In [ ]:
for i in range(len(submit)):
    pred_label=ftmodel.predict(submit['text'][i])
    print(pred_label)
    pred = pred_label[0][0]
    print(pred)
    #if(pred == '__label__notadisaster'):
     #     plabel = 0
    #else:
     #     plabel = 1
    #targets.append(plabel)
print(targets)    
len(targets)

In [ ]:
targets

In [ ]:
submit['target']=targets
#submit=submit.drop(['text'],axis=1)
#submit.to_csv('submission.csv')
submit.head(20)

In [ ]:
sample_submission = pd.read_csv(os.path.join(path,'sample_submission.csv'))

In [ ]:
sample_submission['target'] = targets
sample_submission.head()
len(sample_submission)

In [3]:
embeddings_df = pd.read_pickle('/kaggle/input/fasttext-twitter-derived-embeddings/twitter_derived_embeddings')
embeddings_df.head()

,Token,Embeddings
0,aa,"[0.06992905, -0.6443515, 0.32829714, -0.420536..."
1,aaaa,"[-0.29368854, -0.28680354, 0.082138516, -0.093..."
2,aaaaaaallll,"[0.21100129, -0.24272849, -0.15285794, -0.2225..."
3,aaaaaand,"[0.15002657, 0.121883154, -0.13981317, 0.04672..."
4,aaarrrgghhh,"[0.23939471, -0.120745406, -0.023043174, -0.16..."


In [4]:
EMBEDDING_DIM = 300

fasttext_embedding_idx = {}
for idx, row in embeddings_df.iterrows():
    word = row[0]
    embeddings = np.asarray(row[1], 'float32')
    fasttext_embedding_idx[word] = embeddings

# print only 20
fasttext_embedding_idx['earthquake'][:20]

array([ 0.36360028,  0.06347924, -0.20796725, -0.1053633 , -0.4619699 ,
       -0.5379099 , -0.24516329, -0.373171  ,  0.55195075, -0.0964388 ,
       -0.02738901, -0.68742454,  0.04655449,  0.31652683, -0.28420547,
        0.32772318, -0.48024014, -0.5669692 ,  0.5986041 , -0.4811064 ],
      dtype=float32)

In [5]:
fasttext_embedding_idx['volcano'][:20]

array([ 0.3942571 , -0.00165699,  0.21576007, -0.6281674 , -0.61796415,
        0.5220277 , -0.10654958, -0.51835054,  0.11060573, -0.45126608,
       -0.65426755, -0.5152729 , -0.17358208,  0.15663569, -0.18588951,
        0.26260749, -0.5199955 , -0.48783213,  0.64488477, -0.32145846],
      dtype=float32)